In [2]:
 %load_ext sql

In [3]:
%sql sqlite:///library.db

'Connected: @library.db'

In [21]:
%%sql
PRAGMA foreign_keys = OFF;

DROP TABLE IF EXISTS CD;
DROP TABLE IF EXISTS CD_Detail;
DROP TABLE IF EXISTS Books;
DROP TABLE IF EXISTS Book_Detail;
DROP TABLE IF EXISTS Scientific_Journals;
DROP TABLE IF EXISTS SJ_Detail;
DROP TABLE IF EXISTS Magazines;
DROP TABLE IF EXISTS Magazine_Detail;
DROP TABLE IF EXISTS Items;
DROP TABLE IF EXISTS Personnels;
DROP TABLE IF EXISTS Users;
DROP TABLE IF EXISTS People;
DROP TABLE IF EXISTS Borrowing;
DROP TABLE IF EXISTS Request;
DROP TABLE IF EXISTS Request_by;
DROP TABLE IF EXISTS Events;
DROP TABLE IF EXISTS Event_Detail;
DROP TABLE IF EXISTS Hosted_by;
DROP TABLE IF EXISTS Joining;

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [22]:
%%sql

PRAGMA foreign_keys = ON;

/*****************
*	Items
*****************/
CREATE TABLE IF NOT EXISTS Items(
    id INT PRIMARY KEY,
    availability VARCHAR(10) DEFAULT 'available',
    CHECK (availability = 'available' 
		OR availability = 'borrowed' 
		OR availability = 'coming')
);


CREATE TABLE IF NOT EXISTS CD(
	id INT PRIMARY KEY,
	ISRC VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE,
    FOREIGN KEY (ISRC) REFERENCES CD_Detail(ISRC) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS CD_Detail(
	ISRC VARCHAR(50) PRIMARY KEY,
    title VARCHAR(50) NOT NULL,
    releaseDate DATE NOT NULL,
	artist VARCHAR(50) NOT NULL,
	studio VARCHAR(50) NOT NULL,
    genre VARCHAR(30)
);

CREATE TABLE IF NOT EXISTS Magazines(
	id INT PRIMARY KEY,
	ISSN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE,
    FOREIGN KEY (ISSN) REFERENCES Magazine_Detail(ISSN) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Magazine_Detail(
	ISSN VARCHAR(50) PRIMARY KEY,
    title VARCHAR(50) NOT NULL,
    releaseDate DATE NOT NULL,
	publisher VARCHAR(50) NOT NULL,
    genre VARCHAR(30)
);

CREATE TABLE IF NOT EXISTS Scientific_Journals(
	id INT PRIMARY KEY,
	ISSN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE,
    FOREIGN KEY (ISSN) REFERENCES SJ_Detail(ISSN) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS SJ_Detail(
	ISSN VARCHAR(50) PRIMARY KEY,
    title VARCHAR(50) NOT NULL,
    releaseDate DATE NOT NULL,
	field VARCHAR(30) NOT NULL,
	researcher VARCHAR(50) NOT NULL
);

CREATE TABLE IF NOT EXISTS Books(
	id INT PRIMARY KEY,
	ISBN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE,
    FOREIGN KEY (ISBN) REFERENCES Book_Detail(ISBN) ON DELETE CASCADE
);

CREATE TABLE Book_Detail(
	ISBN VARCHAR(50) PRIMARY KEY,
    title VARCHAR(50) NOT NULL,
    releaseDate DATE NOT NULL,
	author VARCHAR(50) NOT NULL,
    publisher VARCHAR(50) NOT NULL,
	genre VARCHAR(30) NOT NULL,
	type VARCHAR(7) NOT NULL,
	CHECK (type = 'printed' 
		OR type = 'online')
);

/*****************
*	People
*****************/
CREATE TABLE IF NOT EXISTS People (
	pid INT PRIMARY KEY,
	firstName VARCHAR(50) NOT NULL,
	lastName VARCHAR(50) NOT NULL,
	email VARCHAR(50),
	phone VARCHAR(10),
    address VARCHAR(256) NOT NULL,
    CHECK (email LIKE '%@%')
);

CREATE TABLE IF NOT EXISTS Users (
	pid INT PRIMARY KEY,
	totalFine INT DEFAULT 0,
	FOREIGN KEY (pid) REFERENCES People(pid) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Personnels (
	pid INT PRIMARY KEY,
	position VARCHAR(30) NOT NULL,
    salary REAL,
	FOREIGN KEY (pid) REFERENCES People(pid) ON DELETE CASCADE
);

/*****************
*	Event
*****************/
CREATE TABLE IF NOT EXISTS Events (
	eid INT PRIMARY KEY,
	event INT NOT NULL,
    room VARCHAR(30) NOT NULL,
	eventDate DATE,
	eventTime VARCHAR(50),
    FOREIGN KEY (event) REFERENCES Event_Detail(event) ON DELETE CASCADE
    
);

CREATE TABLE IF NOT EXISTS Event_Detail (
    event INT PRIMARY KEY,
    name VARCHAR (50) NOT NULL,
	type VARCHAR (30) NOT NULL,
    audience VARCHAR(30) NOT NULL,
	description VARCHAR(300)
);


/****************
*	Request
*****************/
CREATE TABLE IF NOT EXISTS Request (
	rid INT PRIMARY KEY,
	description VARCHAR(300)
);

/*****************
*	Relationships
*****************/
CREATE TABLE IF NOT EXISTS Borrowing (
	id INT,
	uid INT,
	dateBorrowed DATE,
    dateReturned DATE,
	dueDate DATE,
	fine INT DEFAULT 0,
	PRIMARY KEY (id, uid),
	FOREIGN KEY (id) REFERENCES Items(id)
	ON DELETE CASCADE,
	FOREIGN KEY (uid) REFERENCES Users(pid)
	ON DELETE CASCADE,
	CHECK (dateBorrowed < dueDate 
		OR dateBorrowed < dateReturned)
);

CREATE TABLE IF NOT EXISTS Request_by (
	rid INT,
	pid INT,
	PRIMARY KEY(rid,pid),
	FOREIGN KEY (rid) REFERENCES Request(rid)
	ON DELETE CASCADE,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Hosted_by (
	eid INT,
	pid INT,
	PRIMARY KEY(eid,pid),
	FOREIGN KEY (eid) REFERENCES Events(eid)
	ON DELETE CASCADE,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Joining (
	eid INT,
	pid INT,
	PRIMARY KEY(pid, eid),
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE,
	FOREIGN KEY (eid) REFERENCES Events(eid)
	ON DELETE CASCADE
);



 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [23]:
%%sql

/*****************
*	Triggers
*****************/
--When inserted to the Borrowing table, sets dateBorrowed and dueDate when not set
CREATE TRIGGER IF NOT EXISTS borrowDate AFTER INSERT ON Borrowing
FOR EACH ROW
WHEN (NEW.dateBorrowed IS NULL)
BEGIN 
    UPDATE Borrowing SET dateBorrowed = DATE('now') WHERE id = NEW.id;
    UPDATE Borrowing SET dueDATE = DATE('now','+14 day') WHERE dueDate IS NULL AND id = NEW.id;
END;


CREATE TRIGGER IF NOT EXISTS fine AFTER UPDATE ON Borrowing
FOR EACH ROW
WHEN (NEW.dateReturned IS NOT NULL AND NEW.dueDate IS NOT NULL AND NEW.dateReturned > NEW.dueDate)
BEGIN
    UPDATE Borrowing SET fine = JulianDay(NEW.dateReturned) - JulianDay(NEW.dueDate) WHERE id = NEW.id;
    UPDATE Users SET totalFine = totalFine + (JulianDay(NEW.dateReturned) - JulianDay(NEW.dueDate));
END;


 * sqlite:///library.db
Done.
Done.


[]

In [47]:
%%sql

/*****************
*	Insert to test trigger
*****************/

INSERT INTO Borrowing(id, uid) VALUES (1,11);
UPDATE Borrowing SET dateReturned = DATE('2020-04-11') WHERE id = 1;

 * sqlite:///library.db
Done.
1 rows affected.


[]

In [24]:
%%sql
/*****************
*	Insertion
*****************/
/*****************
*	For Items
*****************/

INSERT INTO Items (id, availability) 
    VALUES (1, 'coming'),
           (2, 'borrowed'),
           (3, 'available'),
           (4, 'available'),
           (5, 'available'),
           (6, 'coming'),
           (7, 'coming'),
           (8, 'available'),
           (9, 'coming'),
           (10, 'borrowed'),
           (11, 'coming'),
           (12, 'available'),
           (13, 'coming'),
           (14, 'coming'),
           (15, 'available'),
           (16, 'coming'),
           (17, 'coming'),
           (18, 'available'),
           (19, 'available'),
           (20, 'available'),
           (21, 'coming'),
           (22, 'available'),
           (23, 'borrowed'),
           (24, 'available'),
           (25, 'available'),
           (26, 'coming'),
           (27, 'available'),
           (28, 'available'),
           (29, 'borrowed'),
           (30, 'available'),
           (31, 'coming'),
           (32, 'coming'),
           (33, 'available'),
           (34, 'coming'),
           (35, 'available'),
           (36, 'borrowed'),
           (37, 'available'),
           (38, 'available'),
           (39, 'borrowed'),
           (40, 'borrowed');

INSERT INTO CD_Detail (ISRC, title, releaseDate, artist, studio, genre) 
    VALUES ('US-B11-13-00005', 'CD1', '2016-01-01','Billy Bob', 'Studio1', 'Rock'),
           ('US-B20-13-00004', 'CD2', '2005-07-07','Artist Two', 'Studio2', 'R&B'),
           ('US-B01-13-00002', 'CD3', '2005-05-09','Billy Bab', 'Studio3', 'R&B'),
           ('US-B14-99-00105', 'CD4', '2009-06-01','Antonio Kim', 'Studio4', 'Rock'),
           ('UK-B07-13-00001', 'CD5', '2014-02-22','Nicholas Mang', 'Studio5', 'R&B'),
           ('US-B09-13-00011', 'CD6', '1999-09-30','Banana Man', 'Studio6', 'Rock'),
           ('UK-B05-01-00234', 'CD7', '1988-08-15','Eve Mina', 'Studio7', 'Rock'),
           ('UK-B03-13-00033', 'CD8', '2017-10-01','Mary Lee', 'Studio8', 'R&B'),
           ('UK-B11-03-00123', 'CD9', '2001-04-21','Artist Nine', 'Studio9', 'R&B'),
           ('US-B21-15-00012', 'CD10', '2002-08-19','Ocean Mama', 'Studio10', 'R&B');

INSERT INTO CD (id, ISRC) 
    VALUES (2, 'US-B11-13-00005'),
           (6, 'US-B20-13-00004'),
           (10, 'US-B01-13-00002'),
           (16, 'US-B14-99-00105'),
           (17, 'UK-B07-13-00001'),
           (25, 'US-B09-13-00011'),
           (32, 'UK-B05-01-00234'),
           (33, 'UK-B03-13-00033'),
           (37, 'UK-B11-03-00123'),
           (38, 'US-B21-15-00012');
        
INSERT INTO Magazine_Detail (ISSN, title, releaseDate, publisher, genre) 
    VALUES ('1234-4321', 'Magazine1', '2010-08-14','Publisher1','Fashion'),
           ('1271-4391', 'Magazine2', '2019-10-02','Publisher2','Sports'),
           ('1294-4102', 'Magazine3', '2013-11-21','Publisher3','Video Games'),
           ('1234-3105', 'Magazine4', '2020-01-02','Publisher4','Cooking'),
           ('0713-0001', 'Magazine5', '2012-12-20','Publisher5','Sports'),
           ('0913-7811', 'Magazine6', '2011-10-06','Publisher6','Gardening'),
           ('0501-9023', 'Magazine7', '1998-11-04','Publisher7','Video Games'),
           ('0313-8233', 'Magazine8', '2009-03-19','Publisher8','Video Games'),
           ('1103-0123', 'Magazine9', '2000-12-01','Publisher9','Fashion'),
           ('2115-1277', 'Magazine10', '2017-10-10','Publisher10','Video Games'); 

INSERT INTO Magazines (id, ISSN) 
    VALUES (11, '1234-4321'),
           (13, '1271-4391'),
           (19, '1294-4102'),
           (20, '1234-3105'),
           (24, '0713-0001'),
           (26, '0913-7811'),
           (28, '0501-9023'),
           (34, '0313-8233'),
           (36, '1103-0123'),
           (39, '2115-1277');  

INSERT INTO SJ_Detail (ISSN, title, releaseDate, field, researcher) 
    VALUES ('4321-4321', 'Scientific_Journal1', '2001-06-29', 'Agriculture', 'Angus Kan'),
           ('4391-2036', 'Scientific_Journal2', '2003-07-04', 'Physics', 'Billy Jeans'),
           ('1354-4102', 'Scientific_Journal3', '1998-06-16', 'Physics', 'James Chin'),
           ('2235-3445', 'Scientific_Journal4', '2002-01-02', 'Chemistry', 'Pam Beasley'),
           ('3875-4968', 'Scientific_Journal5', '2019-10-09', 'Agriculture', 'Michael Wang'),
           ('1357-5555', 'Scientific_Journal6', '2004-10-31', 'Chemistry', 'Dwight Schrute'),
           ('1232-9023', 'Scientific_Journal7', '2002-10-23', 'Agriculture', 'Dean Yonata'),
           ('1123-2069', 'Scientific_Journal8', '1999-03-02', 'Chemistry', 'Meme Man'),
           ('1112-5040', 'Scientific_Journal9', '2010-08-23', 'Chemistry', 'Robbie Kailley'),
           ('5995-0044', 'Scientific_Journal10', '1999-12-12', 'Botany', 'Robert Key'); 

INSERT INTO Scientific_Journals (id, ISSN) 
    VALUES (5, '4321-4321'),
           (7, '4391-2036'),
           (8, '1354-4102'),
           (12, '2235-3445'),
           (15, '3875-4968'),
           (21, '1357-5555'),
           (22, '1232-9023'),
           (30, '1123-2069'),
           (35, '1112-5040'),
           (40, '5995-0044'); 

INSERT INTO Book_Detail (ISBN, title, releaseDate, author, publisher, genre, type) 
    VALUES ('978-90-55687-02-5', 'Book1', '2017-10-01', 'Erwin Frogo', 'Publisher1', 'Mystery', 'online'),
           ('111-92-92545-12-4', 'Book2', '1998-09-25', 'Jenkins Kenker', 'Publisher2', 'Romance', 'online'),
           ('264-02-19075-03-6', 'Book3', '1984-10-01', 'Mem Loot', 'Publisher3', 'Mystery', 'printed'),
           ('853-98-88559-25-5', 'Book4', '2007-06-08', 'Rooty Rao', 'Publisher4', 'Mystery', 'printed'),
           ('886-10-10293-88-3', 'Book5', '2017-10-01', 'Nongo Bongo', 'Publisher5', 'Romance', 'online'),
           ('091-12-57483-70-5', 'Book6', '2020-01-20', 'Chingus Goo', 'Publisher6', 'Romance', 'online'),
           ('102-88-13579-93-0', 'Book7', '2013-10-10', 'Bumbo Bag', 'Publisher7', 'Fantasy', 'online'),
           ('345-23-25468-94-2', 'Book8', '1987-05-22', 'Mama Mio', 'Publisher8', 'Adventure', 'online'),
           ('398-45-85643-19-5', 'Book9', '2017-12-25', 'Jason Lim', 'Publisher9', 'Adventure', 'printed'),
           ('767-04-31288-59-1', 'Book10', '1960-03-10', 'Lovu Yoo', 'Publisher10', 'Romance', 'printed');
        
INSERT INTO Books (id, ISBN) 
    VALUES (1, '978-90-55687-02-5'),
           (3, '111-92-92545-12-4'),
           (4, '264-02-19075-03-6'),
           (9, '853-98-88559-25-5'),
           (14, '886-10-10293-88-3'),
           (18, '091-12-57483-70-5'),
           (23, '102-88-13579-93-0'),
           (27, '345-23-25468-94-2'),
           (29, '398-45-85643-19-5'),
           (31, '767-04-31288-59-1');

INSERT INTO People (pid, firstName, lastName, email, phone, address)
    VALUES (1, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC'),
           (2, 'Anna', 'Anderson', 'aayums@gmail.com', '604-876-5640', '1333 Robson St, Vancouver'),
           (3, 'Will', 'Berenger', 'willb@sfu.ca', '778-828-8102', '501 Dunsmuir St, Vancouver, BC'),
           (4, 'Dequarius', 'McGregor', 'nelkdm135@gmail.com', '778-654-3210', '740 Denman St, Vancouver, BC'),
           (5, 'Ben', 'Daver', 'bd2001@hotmail.com', '778-735-9158', '780 Bidwell St, Vancouver, BC'),
           (6, 'William', 'McMeat', 'wm123haha@hotmail.com', '604-842-8625', '558 W Broadway, Vancouver, BC'),
           (7, 'Angus', 'Kan', 'angus.k@gmail.com', '778-932-5000', '4518 E Hastings St, Burnaby, BC'),
           (8, 'Bonnie', 'Chan', 'bonniechan@gmail.com', '604-789-4561', '1479 Robson St, Vancouver, BC'),
           (9, 'Happy', 'Lee', 'happymonkey@gmail.com', '604-930-8401', '223 Georgia St, Vancouver, BC'),
           (10, 'Mary', 'Young', 'maryhaha@hotmail.com', '604-601-6010', '270 Robson St, Vancouver, BC'),
           (11, 'Antonio', 'Kim', 'akim123@gmail.com', '604-684-2000', '1610 Robson St, Vancouver, BC'),
           (12, 'Ann', 'Sheek', 'ansheek3@hotmail.com', '778-998-6655', '5695 Victoria Dr, Vancouver, BC'),
           (13, 'May', 'Wongers', 'maywonger343@hotmail.com', '778-888-8880', '841 Bidwell St, Vancouver, BC'),
           (14, 'Rachel', 'White', 'rwhite12@gmail.com', '778-820-8639', '3420 Cambie St, Vancouver, BC'),
           (15, 'Nicole', 'Chan', 'nico321@gmail.com', '778-401-7412', '5083 Victoria Dr, Vancouver, BC'),
           (16, 'Sako', 'Moody', 'sako.moody@gmail.com', '604-605-9435', '1150 Davie St, Vancouver, BC'),
           (17, 'Leanne', 'Wong', 'lwong@hotmail.com', '604-803-6019', '4131 No 5 Rd, Richmond, BC'),
           (18, 'Mina', 'Yamaguchi', 'foodyama@hotmail.com', '778-800-1323', '225 Robson St, Vancouver, BC'),
           (19, 'Coco', 'Puff', 'cocoyoyo@gmail.com', '778-987-3356', '551 Howe St, Vancouver, BC'),
           (20, 'Kong', 'Gee', 'yummygee@gmail.com', '604-816-5521', '8611 Alexandra Rd, Richmond, BC'),
           (21, 'Arash', 'Oboodi', 'oboodi9817@gmail.com', '778-778-1000', '4580 No 3 Rd #2065, Richmond, BC'),
           (22, 'Alex', 'Do', 'doitalready@hotmail.com', '778-100-1069', '3379 Sexsmith Rd #1123, Richmond, BC'),
           (23, 'Francis', 'Chan', 'frchan@gmail.com', '604-420-5521', '115-5668 Hollybridge Way, Richmond, BC'),
           (24, 'Jennifer', 'Brown', 'brownie.jen@hotmail.com', '604-558-9712', '3486 Kingsway, Vancouver, BC'),
           (25, 'Ashley', 'Ho', 'hoho123@gmail.com', '778-445-3328', '4501 North Rd #203, Burnaby, BC');

INSERT INTO Users (pid, totalFine)
    VALUES (1, 0),
           (2, 0),
           (6, 0),
           (8, 2),
           (9, 5),
           (10, 1),
           (11, 3),
           (12, 0),
           (16, 0),
           (18, 6);

INSERT INTO Personnels (pid, position, salary)
    VALUES (3, 'Librarian', 3600.00),
           (4, 'Library Director', 4800.00),
           (5, 'Assistant Librarian', 2500.00),
           (7, 'Librarian', 3400.00),
           (13, 'Library Clerk', 2800.00),
           (14, 'Volunteer', 0.00),
           (15, 'Librarian', 3600.00),
           (17, 'Volunteer', 0.00),
           (19, 'Library Technician', 3800.00),
           (20, 'Volunteer', 0.00);
        
INSERT INTO Borrowing (id, uid, dateBorrowed, dateReturned, dueDate, fine) 
    VALUES (2, 1, '2020-01-06', '2020-01-11', '2020-01-20', 0),
           (5, 1, '2020-01-06', '2020-01-11', '2020-01-20', 0),
           (8, 8, '2019-11-12', '2019-11-28', '2019-11-26', 2),
           (23, 9, '2020-03-03', '2020-03-20', '2020-03-17', 3),
           (11, 9, '2019-12-01', '2019-12-17', '2019-12-15', 2),
           (20, 10, '2020-01-11', '2020-01-26', '2020-01-25', 1),
           (28, 11, '2019-09-12', '2019-09-28', '2019-09-26', 2),
           (3, 11, '2020-02-02', '2020-02-17', '2020-01-16', 1),
           (13, 12, '2019-10-26', '2019-10-28', '2019-11-09', 0),
           (34, 18, '2020-02-02', '2020-02-22', '2020-02-16', 6);
        
INSERT INTO Request (rid, description) 
    VALUES (1, 'Where is Book2?'),
           (2, 'What is the most popular CD right now?'),
           (3, 'Where is the closest washroom?'),
           (4, 'I love cooking, so what is good cooking book?'),
           (5, 'Can I ask where is the scientific journal section?'),
           (6, 'How long can I borrow this CD for?'),
           (7, 'How long can I borrow this magazine for?'),
           (8, 'Any mystery novel recommendations?'),
           (9, 'How much will I get fined if I return an item after the due date?'),
           (10, 'Any romance novel recommendations?');

INSERT INTO Request_by (pid, rid) 
    VALUES (1, 1),
           (2, 4),
           (6, 9),
           (8, 2),
           (11, 3),
           (11, 5),
           (11, 8),
           (12, 6),
           (12, 7),
           (16, 10);

INSERT INTO Event_Detail (event, name, type, audience, description)
    VALUES (1, 'Monday Evening Book Club', 'Book Club Meeting', 'Adults', 'Come join us for weekly Monday Book Club'),
           (2, 'English Practice Group', 'Language Learning', 'Anyone', 'Interested in learning English, register for free and join'),
           (3, 'Art Show 1', 'Art', 'Anyone interested in Art', 'Come join us at Art Show 1'),
           (4, 'Japanese Practice Group', 'Language Learning', 'Anyone', 'Interested in learning English, register for free and join'),
           (5, 'Friday Evening Book Club', 'Book Club Meeting', 'Adults', 'Come join us for weekly Friday Book Club'),
           (6, 'Polar Express Night', 'Movie Night', 'Kids', 'Come join us for some hot chocolate and adventure to North Pole'),
           (7, 'Saturday Morning Book Club', 'Book Club Meeting', 'Adults', 'Come join us for weekly Saturday Book Club'),
           (8, 'Kids Coding Camp', 'Programming', 'Grade 3-7', 'Come join us at Coding Camp hosted by Quack'),
           (9, 'Chess Day', 'Games', 'Seniors', 'Come join us at this chess event'),
           (10, 'Job Skills: Using Linkedin', 'Career & Employment', 'Adults', 'Want to brush up on some career skills? Join us at this event');       
        
INSERT INTO Events (eid, event, room, eventDate, eventTime) 
    VALUES (1, 1, 'Room 354', '2020-01-13', '17:00-19:00'),
           (2, 1, 'Room 354', '2020-01-20', '17:00-19:00'),
           (3, 2, 'Room 310', '2020-01-17', '18:00-20:00'),
           (4, 3, 'Room 225', '2020-01-20', '17:00-20:00'),
           (5, 4, 'Room 128', '2020-02-22', '10:00-13:00'),
           (6, 6, 'Room 275', '2019-12-20', '10:00-13:00'),
           (7, 7, 'Room 354', '2020-01-11', '09:00-11:00'),
           (8, 7, 'Room 354', '2020-01-18', '09:00-11:00'),
           (9, 8, 'Room 300', '2020-03-10', '10:00-16:00'),
           (10, 10, 'Room 100', '2020-01-10', '11:00-13:00');
    
INSERT INTO Hosted_by (eid, pid) 
    VALUES (8, 19),
           (10, 19),
           (1, 21),
           (5, 21),
           (7, 21),
           (2, 22),
           (4, 22),
           (3, 23),
           (9, 23),
           (6, 24);

INSERT INTO Joining (eid, pid)
    VALUES (1, 1),
           (1, 6),
           (1, 25),
           (2, 11),
           (3, 1),
           (4, 11),
           (5, 9),
           (5, 25),
           (6, 6),
           (9, 9);

 * sqlite:///library.db
Done.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
25 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.


[]

In [29]:
%%sql
select * FROM CD_Detail;


 * sqlite:///library.db
Done.


ISRC,title,releaseDate,artist,studio,genre
US-B11-13-00005,CD1,2016-01-01,Billy Bob,Studio1,Rock
US-B20-13-00004,CD2,2005-07-07,Artist Two,Studio2,R&B
US-B01-13-00002,CD3,2005-05-09,Billy Bab,Studio3,R&B
US-B14-99-00105,CD4,2009-06-01,Antonio Kim,Studio4,Rock
UK-B07-13-00001,CD5,2014-02-22,Nicholas Mang,Studio5,R&B
US-B09-13-00011,CD6,1999-09-30,Banana Man,Studio6,Rock
UK-B05-01-00234,CD7,1988-08-15,Eve Mina,Studio7,Rock
UK-B03-13-00033,CD8,2017-10-01,Mary Lee,Studio8,R&B
UK-B11-03-00123,CD9,2001-04-21,Artist Nine,Studio9,R&B
US-B21-15-00012,CD10,2002-08-19,Ocean Mama,Studio10,R&B


In [46]:
%%sql
select * from Users;


 * sqlite:///library.db
Done.


pid,totalFine
123,10
